## PREPROCESSING

### Install and Importing Library/Package

In [3]:
# Installing Library / Packages
%pip install tensorflow==2.17.1


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

%pip install lime

In [ ]:
%pip install gensim

In [4]:
import tensorflow
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))
print(tf.__version__)

2024-11-24 06:14:40.194562: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 06:14:40.197895: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 06:14:40.206980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 06:14:40.221880: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 06:14:40.226105: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 06:14:40.237700: I tensorflow/core/platform/cpu_feature_gu

Num GPUs Available:  0


NameError: name 'tf' is not defined

In [ ]:
import pandas as pd
import gensim
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report

#Lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from lime.lime_text import IndexedString,IndexedCharacters
from lime.lime_base import LimeBase


### Membuat Model

#### Mengimport file hasil preprocessing

In [ ]:
# Load CSV files into DataFrames
data_train = pd.read_csv('../datasets/output_preprocess/data_train.csv')
data_test = pd.read_csv('../datasets/output_preprocess/data_test.csv')
data_val = pd.read_csv('../datasets/output_preprocess/data_val.csv')

In [ ]:
#Splitting the text from the labels
X_train = data_train['text']
y_train = data_train['label']

X_test = data_test['text']
y_test = data_test['label']

X_val = data_val['text']
y_val = data_val['label']

#### Encode Labels

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)

In [ ]:
#Convert the class vector (integers) to binary class matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [ ]:
MAX_LENGTH = 500

# Create a tokenizer
tokenizer = Tokenizer(oov_token='UNK')
# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train)
# Convert text to sequences of tokens
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)
# Pad sequences to ensure equal length
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, truncating='pre')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, truncating='pre')
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LENGTH, truncating='pre')

In [ ]:
# Train Word2Vec model
sentences = [text.split() for text in X_train]
w2v_model = gensim.models.Word2Vec(sentences, vector_size=200, window=5, min_count=5, workers=4)

# Create embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 200))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

#### Membangun arsitekteur NN

In [ ]:
# mendefinisikan vocabSize
vocabSize = len(tokenizer.word_index) + 1  # +1 for the padding token or OOV token

model = Sequential()
model.add(Embedding(vocabSize, 200, weights=[embedding_matrix])) # Use X_train_pad to get shape
model.add(Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(6, activation='softmax'))


adam = Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()

In [ ]:
#to stop the training when the loss starts to increase
callback = EarlyStopping(
    monitor="val_loss",      # Memantau loss pada data validasi
    patience=4,              # Menunggu 4 epoch sebelum menghentikan pelatihan jika val_loss tidak membaik
    restore_best_weights=True  # Mengembalikan bobot terbaik saat pelatihan dihentikan
)

In [ ]:
# Fit model
history = model.fit(X_train_pad,
                    y_train,
                    validation_data=(X_val_pad, y_val),
                    verbose=1,
                    batch_size=256,
                    epochs=10,
                    callbacks=[callback]
                   )

In [ ]:
plot_model(model, show_shapes=True, dpi=80)

In [ ]:
#print the overall loss and accuracy
model.evaluate(X_test_pad, y_test)

In [ ]:
data_test['label'].value_counts()

In [ ]:
predicted = model.predict(X_test_pad)
y_pred = predicted.argmax(axis=-1)

print(classification_report(le.transform(data_test['label']), y_pred))

In [ ]:
# Visualize Loss & Accuracy

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('../model/emotion-classification.h5')

### Predicting diary input

In [9]:
# Import function for predicting
import function as fc

In [14]:
mytext = "i sad384y593"

#alt+z to wrap text
mytext2 = "Today was one of those days that filled me with pure joy. The weather was perfect, with clear skies and a gentle breeze that made everything feel fresh and new. I woke up early, feeling energized, and decided to take a walk around the neighborhood. The sunrise painted the sky with soft pinks and oranges, and I couldn’t help but pause and take it all in. There’s something so peaceful about those quiet mornings, just me and the world waking up. Later, I met up with a friend for coffee, and we spent hours laughing and reminiscing. It’s amazing how the simplest conversations can fill you with so much happiness, and I realized that joy isn’t about grand moments—it’s found in the small, everyday things. Whether it’s the feeling of the sun on your skin, the sound of a good friend’s laughter, or the peace that comes from just being present, today reminded me that joy is all around if we take the time to appreciate it. I ended the day feeling grateful for everything I have, from the simple moments to the wonderful people in my life. It was one of those days that will stay with me for a long time, a reminder that happiness doesn’t have to be complicated—it’s right here in front of us"

In [16]:
fc.predicting_input(mytext)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step


'The predicted label is: sadness'

In [17]:
fc.predicting_input(mytext2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 596ms/step


'The predicted label is: love'